In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [92]:
df = pd.read_csv('card_cust.csv')

In [96]:
df.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0
2,10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,12.0,7500.0,622.066742,627.284787,0.000000,12.0
3,10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1.0,1.0,7500.0,0.000000,1297.116322,0.000000,12.0
4,10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0.0,1.0,1200.0,678.334763,244.791237,0.000000,12.0


In [94]:
df['MINIMUM_PAYMENTS'].mean()

1297.116321697626

In [95]:
df['CREDIT_LIMIT'] = df['CREDIT_LIMIT'].fillna(df['CREDIT_LIMIT'].mean())
df['MINIMUM_PAYMENTS'] = df['MINIMUM_PAYMENTS'].fillna(df['MINIMUM_PAYMENTS'].mean())

In [8]:
#Q1 문제 제대로 읽기

In [17]:
#round(df[['BALANCE','TENURE','CREDIT_LIMIT']].corr(method='pearson').replace(1,np.nan).max().max(),2)

0.47

In [106]:
round(df.groupby('TENURE')[['BALANCE','CREDIT_LIMIT']].corr(method = 'pearson').replace(1,np.nan).max().max(),2)

0.95

In [134]:
df_corr = df.groupby('TENURE')[['BALANCE','CREDIT_LIMIT']].corr()
df_corr.xs('BALANCE',level=1)['CREDIT_LIMIT'].round(2).max()

0.95

In [18]:
#Q2

In [35]:
#step1
scaler = StandardScaler()
df_std = scaler.fit_transform(df.iloc[:,1:])

In [36]:
df_std

array([[-0.84876759, -0.41987944, -0.4419358 , ..., -0.44372465,
        -0.46554357,  0.28242902],
       [ 0.28279099,  0.01213096, -0.4690169 , ..., -0.08615941,
         0.33159169,  0.28242902],
       [ 0.0296341 ,  0.44414137, -0.24953794, ..., -0.25675456,
        -0.46554357,  0.28242902],
       ...,
       [-0.04555583,  0.44414137, -0.4690169 , ..., -0.1126367 ,
        -0.46554357,  0.28242902],
       [ 0.18482699,  0.44414137, -0.4233367 , ..., -0.07613978,
        -0.46554357,  0.28242902],
       [-0.34079285,  0.44414137, -0.20275918, ...,  0.00909944,
        -0.46554357,  0.28242902]])

In [39]:
#step2
optimal_score = 0
optimal_k = 2
for k in range(2,6):
    kmeans = KMeans(n_clusters=k, random_state = 1234)
    kmeans.fit(df_std)
    labels = kmeans.labels_
    s_score = silhouette_score(df_std, labels) # 이 값이 최대가 되는 것이 optimal
    print(k, s_score)
    if(s_score < optimal_score) :
        optimal_score = s_score
        optimal_k = k

2 0.30752815304560793
3 0.19481269449969035
4 0.20715098494639902
5 0.19379056549331705


In [40]:
optimal_k

2

In [42]:
kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=123)
kmeans_optimal.fit(df_std)
df.loc[:,'cluster'] = kmeans_optimal.labels_

In [47]:
round(df.groupby('cluster')['ONEOFF_PURCHASES'].mean().max(),2)

3946.19

In [48]:
#Q3

In [49]:
4 % 4

0

In [50]:
5%4

1

In [52]:
df = df.drop('cluster',axis = 1)

In [107]:
df_train = df.loc[df['CUST_ID']%4 == 0].drop('CUST_ID',axis = 1)
df_test= df.loc[df['CUST_ID']%4 != 0].drop('CUST_ID',axis = 1)

In [116]:
df_train_temp = df_train.set_index('ONEOFF_PURCHASES').reset_index()
data = df_train_temp.iloc[:,1:]
value = df_train_temp.iloc[:,0]

In [126]:
df_test_temp = df_test.set_index('ONEOFF_PURCHASES').reset_index()
x_test = df_test_temp.iloc[:,1:]
y_test = df_test_temp.iloc[:,0]

In [144]:
model = DecisionTreeRegressor(random_state=1234)
model.fit(data,value)

DecisionTreeRegressor(random_state=1234)

In [145]:
pred = model.predict(x_test)

In [146]:
pred[:4]

array([  0.  ,   0.  , 701.44,  16.5 ])

In [147]:
X= df_train.drop(columns = ['ONEOFF_PURCHASES'])
y = df_train['ONEOFF_PURCHASES']
x_test = df_test.drop(columns = ['ONEOFF_PURCHASES'])

In [151]:
model.fit(X,y)
prednew = model.predict(x_test)
prednew[:4]

array([  0.  ,   0.  , 701.44,  16.5 ])

In [129]:
round(np.sqrt(sum((pred-y_test)**2)/len(pred)),2)

1268.32

In [150]:
from sklearn.metrics import mean_squared_error as MSE
round(MSE(y_true = y_test, y_pred = pred)**0.5,1)

1253.1

In [ ]:
?????????다시해보기